Problem Statement : It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data. How do you tidy it up so you can get on with the analysis?

 Take this monstrosity as the DataFrame to use in the following puzzles: 

 
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 'Budapest_PaRis', 'Brussels_londOn'], 'FlightNumber': [10045, np.nan, 10065, np.nan, 10085], 'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]], 'Airline': ['KLM(!)', '

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN','londON_StockhOlm',
                    'Budapest_PaRis', 'Brussels_londOn'],
                  'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
                  'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                  'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
                  '12. Air France', '"Swiss Air"']})

In [3]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

In [4]:
df['FlightNumber'] = df['FlightNumber'].interpolate().astype(float)

In [5]:
# for i in range(df['FlightNumber'].count()+1):
#       if pd.isnull(df.loc[i,'FlightNumber']):
#           df.loc[i, 'FlightNumber'] = df.loc[i-1, 'FlightNumber'] + 10

In [6]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055.0,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075.0,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


2. The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

In [7]:
df[['From','To']] = df.From_To.str.split("_",expand=True)

In [8]:
df

,From_To,FlightNumber,RecentDelays,Airline,From,To
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!),LoNDon,paris
1,MAdrid_miLAN,10055.0,[],<Air France> (12),MAdrid,miLAN
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. ),londON,StockhOlm
3,Budapest_PaRis,10075.0,[13],12. Air France,Budapest,PaRis
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air""",Brussels,londOn


3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)

In [9]:
for i in df[['From','To']].columns:
  df[i] = df[i].apply(lambda x: x.capitalize())

In [10]:
df

,From_To,FlightNumber,RecentDelays,Airline,From,To
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!),London,Paris
1,MAdrid_miLAN,10055.0,[],<Air France> (12),Madrid,Milan
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,Budapest_PaRis,10075.0,[13],12. Air France,Budapest,Paris
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air""",Brussels,London


4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

In [11]:
df.drop('From_To',axis=1, inplace= True)
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045.0,"[23, 47]",KLM(!),London,Paris
1,10055.0,[],<Air France> (12),Madrid,Milan
2,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075.0,[13],12. Air France,Budapest,Paris
4,10085.0,"[67, 32]","""Swiss Air""",Brussels,London


5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN.

Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [12]:
delays = pd.DataFrame(df['RecentDelays'].to_list(), columns = ['delay_1', 'delay_2', 'delay_3'])
delays

,delay_1,delay_2,delay_3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [13]:
df.drop(['RecentDelays'], inplace=True, axis=1)
df

,FlightNumber,Airline,From,To
0,10045.0,KLM(!),London,Paris
1,10055.0,<Air France> (12),Madrid,Milan
2,10065.0,(British Airways. ),London,Stockholm
3,10075.0,12. Air France,Budapest,Paris
4,10085.0,"""Swiss Air""",Brussels,London


In [14]:
df.insert(loc = 3, column='delay_1' , value=delays['delay_1'])
df.insert(loc = 4, column='delay_2' , value=delays['delay_2'])
df.insert(loc = 5, column='delay_3' , value=delays['delay_3'])
df

,FlightNumber,Airline,From,delay_1,delay_2,delay_3,To
0,10045.0,KLM(!),London,23.0,47.0,NaN,Paris
1,10055.0,<Air France> (12),Madrid,NaN,NaN,NaN,Milan
2,10065.0,(British Airways. ),London,24.0,43.0,87.0,Stockholm
3,10075.0,12. Air France,Budapest,13.0,NaN,NaN,Paris
4,10085.0,"""Swiss Air""",Brussels,67.0,32.0,NaN,London
